## Data Cleaning and Preproccesssing

## Import Libraries

In [119]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## Load the Datasets

In [68]:
opensooq_df = pd.read_csv("opensooq_properties.csv")
bayut_df = pd.read_csv("bayut_detailed.csv")

## Initial Exploration

In [69]:
print("OpenSooq Shape:", opensooq_df.shape)
print("Bayut Shape:", bayut_df.shape)
opensooq_df.head()

OpenSooq Shape: (8155, 7)
Bayut Shape: (1809, 7)


,source,title,price,location,size,listing_type,link
0,OpenSooq,118 m2 3 Bedrooms Apartments for Sale in Musca...,"45,000 OMR",", Bosher, 964439XX",118 m²,Property for Sale,https://om.opensooq.com/en/search/264252105
1,OpenSooq,Building for Sale in Muscat Ghala,"950,000 OMR",", Ghala, 924618XX",2200 m²,Property for Sale,https://om.opensooq.com/en/search/253985415
2,OpenSooq,77 m2 2 Bedrooms Apartments for Sale in Muscat...,"31,000 OMR",", Azaiba, 715560XX",77 m²,Property for Sale,https://om.opensooq.com/en/search/266186647
3,OpenSooq,52 m2 Studio Apartments for Sale in Muscat Al ...,"29,000 OMR",", Al Mawaleh, 964439XX",52 m²,Property for Sale,https://om.opensooq.com/en/search/265496647
4,OpenSooq,320 m2 5 Bedrooms Villa for Sale in Muscat Qur...,"100,000 OMR",", Quriyat, 985055XX",320 m²,Property for Sale,https://om.opensooq.com/en/search/266519035


## Standardize Column Names

In [72]:
opensooq_df.columns = opensooq_df.columns.str.lower().str.replace(" ", "_")
bayut_df.columns = bayut_df.columns.str.lower().str.replace(" ", "_")

## Rename Important Columns for Consistency

In [90]:
# Rename columns for consistency
opensooq_df.rename(columns={'listing_type': 'listing type'}, inplace=True)
bayut_df.rename(columns={'room number': 'room_number'}, inplace=True)

# Add missing column to OpenSooq to match Bayut
opensooq_df['room_number'] = np.nan

# Clean price column
def clean_price(val):
    if pd.isna(val):
        return np.nan
    val = str(val).replace(",", "").replace("OMR", "").strip()
    try:
        return float(val)
    except:
        return np.nan

opensooq_df['price'] = opensooq_df['price'].apply(clean_price)
bayut_df['price'] = bayut_df['price'].apply(clean_price)

# Clean size column
def clean_size(val):
    if pd.isna(val):
        return np.nan
    val = str(val).replace(",", "").replace("m²", "").replace("Sq. M.", "").strip()
    try:
        return float(val)
    except:
        return np.nan

C:\Users\bbuser\AppData\Local\Temp\ipykernel_27264\738106598.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bayut_df.rename(columns={'room number': 'room_number'}, inplace=True)
C:\Users\bbuser\AppData\Local\Temp\ipykernel_27264\738106598.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bayut_df['price'] = bayut_df['price'].apply(clean_price)


In [114]:
# Strip any spaces from column names
opensooq_df.columns = opensooq_df.columns.str.strip()
bayut_df.columns = bayut_df.columns.str.strip()

# Rename columns for consistency
opensooq_df.rename(columns={'area': 'size'}, inplace=True)
bayut_df.rename(columns={'listing_type': 'listing type'}, inplace=True)

# Print columns to confirm
print("OpenSooq columns:", opensooq_df.columns.tolist())
print("Bayut columns:", bayut_df.columns.tolist())

# Add missing column if necessary
if 'room_number' not in opensooq_df.columns:
    opensooq_df['room_number'] = np.nan

# Clean price column
def clean_price(val):
    if pd.isna(val):
        return np.nan
    val = str(val).replace(",", "").replace("OMR", "").strip()
    try:
        return float(val)
    except:
        return np.nan

opensooq_df['price'] = opensooq_df['price'].apply(clean_price)
bayut_df['price'] = bayut_df['price'].apply(clean_price)

# Clean size column
def clean_size(val):
    if pd.isna(val):
        return np.nan
    val = str(val).replace(",", "").replace("m²", "").replace("Sq. M.", "").strip()
    try:
        return float(val)
    except:
        return np.nan

opensooq_df['size'] = opensooq_df['size'].apply(clean_size)
bayut_df['size'] = bayut_df['size'].apply(clean_size)

# Drop 'link' if exists
opensooq_df.drop(columns=['link'], errors='ignore', inplace=True)

# Select only the needed columns
common_columns = ['source', 'title', 'price', 'location', 'room_number', 'size', 'listing type']
opensooq_df = opensooq_df[common_columns]
bayut_df = bayut_df[common_columns]

# Combine
combined_df = pd.concat([opensooq_df, bayut_df], ignore_index=True)

# Convert bedrooms & bathrooms to numeric (if they exist)
for col in ['bedrooms', 'bathrooms']:
    if col in combined_df.columns:
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')

# Extract city from location
combined_df['city'] = combined_df['location'].str.extract(r'([A-Za-z ]+),')

# drop listings without location or price
combined_df.dropna(subset=['price', 'size', 'city'], inplace=True)


# Drop duplicates
combined_df.drop_duplicates(inplace=True)

# Save to CSV
combined_df.to_csv("combined_cleaned_properties.csv", index=False)

# Check result
combined_df.info()


OpenSooq columns: ['source', 'title', 'price', 'location', 'room_number', 'size', 'listing type']
Bayut columns: ['source', 'title', 'price', 'location', 'room_number', 'size', 'listing type']
<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 56 to 1373
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   source        24 non-null     object 
 1   title         24 non-null     object 
 2   price         24 non-null     float64
 3   location      24 non-null     object 
 4   room_number   2 non-null      float64
 5   size          24 non-null     float64
 6   listing type  0 non-null      object 
 7   city          24 non-null     object 
dtypes: float64(3), object(5)
memory usage: 1.7+ KB


In [115]:
# Remove extreme price and size outliers
q_low_price = combined_df['price'].quantile(0.01)
q_high_price = combined_df['price'].quantile(0.99)
q_low_size = combined_df['size'].quantile(0.01)
q_high_size = combined_df['size'].quantile(0.99)

combined_df = combined_df[
    (combined_df['price'] >= q_low_price) & 
    (combined_df['price'] <= q_high_price) &
    (combined_df['size'] >= q_low_size) &
    (combined_df['size'] <= q_high_size)
]

## Feature Engineering

## Generate new features and apply scaling

In [116]:
# Load the cleaned data (optional if already in memory)
combined_df = pd.read_csv("combined_cleaned_properties.csv")

# ─── 1. Create price_per_sqm ─────────────────────────────
combined_df['price_per_sqm'] = combined_df['price'] / combined_df['size']

# ─── 2. Handle missing room numbers ──────────────────────
# Fill missing room numbers with the median
combined_df['room_number'].fillna(combined_df['room_number'].median(), inplace=True)

# ─── 3. One-hot encode 'listing type' ─────────────────────
# One-hot encode 'listing type' and 'city'
df_encoded = pd.get_dummies(combined_df, columns=['listing type', 'city'], drop_first=True)

# Fill missing bedrooms/bathrooms with median
for col in ['bedrooms', 'bathrooms', 'room_number']:
    if col in df_encoded.columns:
        df_encoded[col].fillna(df_encoded[col].median(), inplace=True)

# Create new features
df_encoded['price_per_sqm'] = df_encoded['price'] / df_encoded['size']
df_encoded['price_per_room'] = df_encoded['price'] / (df_encoded['room_number'] + 1)  # avoid div/0


# ─── 4. Drop rows where price_per_sqm is still NaN ────────
df_encoded.dropna(subset=['price_per_sqm'], inplace=True)

# ─── 5. Feature scaling ──────────────────────────────────
features_to_scale = ['price', 'size', 'room_number', 'price_per_sqm']

# a) StandardScaler
standard_scaler = StandardScaler()
df_standard_scaled = df_encoded.copy()
df_standard_scaled[features_to_scale] = standard_scaler.fit_transform(df_encoded[features_to_scale])

# b) MinMaxScaler (optional)
minmax_scaler = MinMaxScaler()
df_minmax_scaled = df_encoded.copy()
df_minmax_scaled[features_to_scale] = minmax_scaler.fit_transform(df_encoded[features_to_scale])
# Features to scale
features_to_scale = ['price', 'size', 'room_number', 'bedrooms', 'bathrooms', 'price_per_sqm', 'price_per_room']

# ─── 6. Preview the result ───────────────────────────────
df_standard_scaled.head()


C:\Users\bbuser\AppData\Local\Temp\ipykernel_27264\2160074355.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['room_number'].fillna(combined_df['room_number'].median(), inplace=True)
C:\Users\bbuser\AppData\Local\Temp\ipykernel_27264\2160074355.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting va

,source,title,price,location,room_number,size,price_per_sqm,city_Al Amarat,city_Al Bustan,city_Al Duqum,...,city_Bidiya,city_Bosher,city_Ghala,city_Jalan Bani Buhassan,city_Mahut,city_Masira,city_Misfah,city_Nizwa,city_Sohar,price_per_room
0,Bayut,Villa,1.735564,"Al Sifah, Muscat",3.464102,-0.390025,-0.149034,False,False,False,...,False,False,False,False,False,False,False,False,False,180.000000
1,Bayut,Residential Land,-0.880043,"Adam, Al Dakhiliya",0.000000,-0.401849,-0.220850,False,False,False,...,False,False,False,False,False,False,False,False,False,28.571429
2,Bayut,Residential Land,-0.749262,"Barka, Al Batinah",0.000000,0.610799,-0.232161,False,False,False,...,False,False,False,False,False,False,False,False,False,40.000000
3,Bayut,Residential Land,1.474003,"Al Bustan, Muscat",0.000000,0.233857,-0.226820,False,True,False,...,False,False,False,False,False,False,False,False,False,234.285714
4,Bayut,Residential Land,2.059245,"Sohar, Al Batinah",0.000000,-0.353075,-0.177933,False,False,False,...,False,False,False,False,False,False,False,False,True,285.428571


## Predictive Modeling

In [120]:
# ─── 1. Define Features and Target ───────────────────────────────
# We'll use Standard Scaled data from Step 3
X = df_standard_scaled.drop(columns=['price'])  # Features
y = df_standard_scaled['price']                # Target: price

In [121]:
# Drop non-numeric / irrelevant columns
X = df_standard_scaled.drop(columns=['price', 'title', 'location', 'source'])
y = df_standard_scaled['price']

In [123]:
# ─── 2. Split the data ───────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ─── 3. Train & Evaluate Models ──────────────────────────────────
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Gardian Boost Regression' :GradientBoostingRegressor(n_estimators=300, learning_rate=0.1, max_depth=5, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    r2 = r2_score(y_test, preds)
    mae = mean_absolute_error(y_test, preds)
    
    print(f"{name}")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE: {mae:.4f}")
    print("-" * 30)


Linear Regression
R² Score: 0.9529
MAE: 0.0904
------------------------------
Decision Tree
R² Score: 0.5329
MAE: 0.3923
------------------------------
Gardian Boost Regression
R² Score: 0.5422
MAE: 0.3769
------------------------------
Random Forest
R² Score: 0.7258
MAE: 0.3506
------------------------------
